## A simple Jupyter notebook to compute various characteristics of a laser cooling scheme
A user needs to input the following information about a scheme.

For a single-$S_1$ scheme:
- lifetime of $S_1$ state (in seconds, s)
- branching ratios from $S_1$ to all $S_2$s (unitless)
- energy differences between $S_1$ and every $S_2$s / laser frequencies addressing transitions between $S_1$ and every $S_2$s (in nanometers, nm)

For a double-$S_1$ scheme:
- lifetimes of two $S_1$ states (in seconds, s)
- branching ratios from the first $S_1$ to all $S_2$s (unitless)
- branching ratios from the second $S_1$ to all $S_2$s (unitless)
- energy differences between the first $S_1$ and every $S_2$s / laser frequencies addressing transitions between $S_1$ and every $S_2$s (in nanometers, nm)
- energy differences between the second $S_1$ and every $S_2$s / laser frequencies addressing transitions between $S_1$ and every $S_2$s (in nanometers, nm)

For both types of schemes:
- molecular mass (in unified atomic mass units, u)
- energy of $S_0$ state (in reciprocal centimeters/wavenumbers, cm$^{-1})

### Needed functions (all described in a paper)

In [ ]:
import numpy as np

In [ ]:
# Scattering rate from Eq. (6)
def inverse_R_single_S1(lambdas, BRs, lifetime):
  sum_inv_lambdas_3 = np.sum(1 / lambdas**3)
  return lifetime * (len(lambdas) + 1) + 0.04160402474381969 * sum_inv_lambdas_3

## For Ne = 1 collapses to the single S1 formula
def inverse_R_double_S1(lambdas, BRs, lifetime, lambdas_prim, BRs_prim, lifetime_prim):
  avg_lifetime = (lifetime + lifetime_prim) / 2
  sum_BRs_over_lambdas_3_BRs = np.sum( (BRs + BRs_prim) / (BRs * lambdas**3 + BRs_prim * lambdas_prim**3) )
  Ne = 2
  Ng = len(lambdas)
  return avg_lifetime * (Ne + Ng) / Ne + 0.04160402474381969 / Ne * sum_BRs_over_lambdas_3_BRs

## Function to invoke single- or double-S1 formulas depending on the input arguments
def inverse_R(lambdas, BRs, lifetime, lambdas_prim = None, BRs_prim = None, lifetime_prim = None):
  if lambdas_prim is None and BRs_prim is None and lifetime_prim is None:
    return inverse_R_single_S1(lambdas, BRs, lifetime)
  else:
    return inverse_R_double_S1(lambdas, BRs, lifetime, lambdas_prim, BRs_prim, lifetime_prim)

In [ ]:
# Initial temperature of molecular gas, estimated from the Boltzmann distribution, from Eq. (12)
def initial_temp(energy_S0):
  return max([4, energy_S0 * 0.62485285866738])

# Closure of a cooling scheme from Eq. (3)
def closure_single_S1(BRs):
  return np.sum(BRs)

## To get double-S1 closure, I'm averaging over two closures, for Ne = 1 also collapses to the single S1 formula
def closure_double_S1(BRs, BRs_prim):
  return (np.sum(BRs) + np.sum(BRs_prim)) / 2

## Function to invoke single- or double-S1 formulas depending on the input arguments
def closure(BRs, BRs_prim = None):
  if BRs_prim is None:
    return closure_single_S1(BRs)
  else:
    return closure_double_S1(BRs, BRs_prim)

# Number of scatterings that retain 10% of the molecules in a bright state from Eq. (4)
def n10(closure):
  return np.log(0.1)/np.log(closure)

In [ ]:
# Number of needed cooling scattering processes from Eq. (9)
def ncool_single_S1(lambdas, BRs, energy_S0, mass, closure):
  sum_of_BR_lambda_ratios = np.sum(BRs / lambdas)
  return np.sqrt(initial_temp(energy_S0) * mass) * closure(BRs) * 0.39579544150466855 / sum_of_BR_lambda_ratios

# Mean photon momentum is an average of mean photon momenta of two subschemes, weighted by the respective closures
def ncool_double_S1(lambdas, BRs, lambdas_prim, BRs_prim, energy_S0, mass):
  sum_of_BR_lambda_ratios_w_closures = np.sum(BRs / lambdas) / closure(BRs) + np.sum(BRs_prim / lambdas_prim) / closure(BRs_prim)
  return 2 * np.sqrt(initial_temp(energy_S0) * mass) * 0.39579544150466855 / sum_of_BR_lambda_ratios_w_closures

## Function to invoke single- or double-S1 formulas depending on the input arguments
def ncool(lambdas, BRs, energy_S0, mass, lambdas_prim = None, BRs_prim = None, lifetime_prim = None):
  if lambdas_prim is None and BRs_prim is None and lifetime_prim is None:
    return ncool_single_S1(lambdas, BRs, energy_S0, mass, closure)
  else:
    return ncool_double_S1(lambdas, BRs, lambdas_prim, BRs_prim, energy_S0, mass)

### For a single-$S_1$ scheme:

In [ ]:
lambdas_A = np.array([1015.1130583331022, 1246.300338439263, 1606.508589363804, 2244.744820493086, 3682.7307769845465])

BRs_A = np.array([0.8066776821737045, 0.005408116361920967, 0.14239486381879604, 0.043026420294922506, 0.002482273056726346]) 

lifetime_A = 0.000010619
mass = 24
energy_S0 = 10.874961

In [ ]:
print("T_init = {:.1f} K \n R^(-1) = {:.2f} um \n n_cool = {:.0f}\n ncool/n10 = {:.4f}\n closure = {:.6f}\n t_cool = {:.1f} ms".format(initial_temp(energy_S0), 
                                                                                         inverse_R(lambdas_A, BRs_A, lifetime_A)*1e6, 
                                                                                         ncool(lambdas_A, BRs_A, lifetime_A, energy_S0, mass), 
                                                                                         ncool(lambdas_A, BRs_A, lifetime_A, energy_S0, mass)/n10(closure(BRs_A)), 
                                                                                         closure(BRs_A),
                                                                                         ncool(lambdas_A, BRs_A, lifetime_A, energy_S0, mass) * inverse_R(lambdas_A, BRs_A, lifetime_A)*1e3))

### For a double-$S_1$ scheme:

lambdas_A = np.array([1015.1130583331022, 1246.300338439263, 1606.508589363804, 2244.744820493086, 3682.7307769845465])
lambdas_B = np.array([1209.604136317404, 1552.8441814677426, 2154.8348542691415, 3483.2314250585123])

BRs_A = np.array([0.8066776821737045, 0.005408116361920967, 0.14239486381879604, 0.043026420294922506, 0.002482273056726346]) 
BRs_B = np.array([0.6786156634845087, 0.28348982187561683, 0.03649184480340057, 0.001397026284591585])

lifetime_A = 0.000010619
lifetime_B = 0.000013134
mass = 24
energy_S0 = 10.874961

In [ ]:
print("T_init = {:.1f} K \n R^(-1) = {:.2f} um \n n_cool = {:.0f}\n ncool/n10 = {:.4f}\n closure = {:.6f}\n t_cool = {:.1f} ms".format(initial_temp(energy_S0), 
                                                                                         inverse_R(lambdas_A, BRs_A, lifetime_A, lambdas_B, BRs_B, lifetime_B)*1e6, 
                                                                                         ncool(lambdas_A, BRs_A, lifetime_A, energy_S0, mass, lambdas_B, BRs_B, lifetime_B), 
                                                                                         ncool(lambdas_A, BRs_A, lifetime_A, energy_S0, mass, lambdas_B, BRs_B, lifetime_B)/n10(closure(BRs_A, BRs_B)), 
                                                                                         closure(BRs_A, BRs_B),
                                                                                         ncool(lambdas_A, BRs_A, lifetime_A, energy_S0, mass, lambdas_B, BRs_B, lifetime_B) * inverse_R(lambdas_A, BRs_A, lifetime_A, lambdas_B, BRs_B, lifetime_B)*1e3))